In [64]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [65]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')

In [66]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [67]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [68]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=0)

In [69]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)


In [70]:
Xtrain = torch.from_numpy(x_train).float()
Xtest = torch.from_numpy(x_test).float()

In [71]:
Xtrain

tensor([[-0.0935,  0.2215],
        [ 1.2217, -0.5342],
        [-1.0655,  0.4104],
        ...,
        [ 1.2789,  0.4104],
        [-0.9512,  0.2215],
        [-1.2943, -1.4316]])

In [72]:
Ytrain = torch.from_numpy(y_train).long()
Ytest = torch.from_numpy(y_test).long()


In [73]:
Ytrain

tensor([1, 1, 1,  ..., 1, 1, 0])

In [74]:
Xtrain.shape, Ytrain.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [75]:
Xtest.shape, Ytest.shape

(torch.Size([311, 2]), torch.Size([311]))

In [76]:
input_size = 2
output_size = 2
hidden_size = 10

In [77]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(input_size,hidden_size)
    self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
    self.fc3 = torch.nn.Linear(hidden_size, output_size)

  def forward(self, X):
    X = torch.relu((self.fc1(X)))
    X = torch.relu((self.fc2(X)))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [78]:
model = Net()

In [79]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.NLLLoss()

In [117]:
epochs = 100

In [118]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Y_pred = model(Xtrain)
  loss = loss_fn(Y_pred, Ytrain)
  loss.backward()
  optimizer.step()
  print("Epochs: ", epoch, " Loss: ", loss.item())


Epochs:  0  Loss:  0.11085053533315659
Epochs:  1  Loss:  0.11063331365585327
Epochs:  2  Loss:  0.11070677638053894
Epochs:  3  Loss:  0.11071518063545227
Epochs:  4  Loss:  0.11056461185216904
Epochs:  5  Loss:  0.11043895035982132
Epochs:  6  Loss:  0.11039023101329803
Epochs:  7  Loss:  0.1104394793510437
Epochs:  8  Loss:  0.11043745279312134
Epochs:  9  Loss:  0.11041850596666336
Epochs:  10  Loss:  0.11019846796989441
Epochs:  11  Loss:  0.11031962186098099
Epochs:  12  Loss:  0.1102529764175415
Epochs:  13  Loss:  0.1101895198225975
Epochs:  14  Loss:  0.11017143726348877
Epochs:  15  Loss:  0.11021310091018677
Epochs:  16  Loss:  0.11020094901323318
Epochs:  17  Loss:  0.11006440967321396
Epochs:  18  Loss:  0.11009106040000916
Epochs:  19  Loss:  0.11002733558416367
Epochs:  20  Loss:  0.1099262684583664
Epochs:  21  Loss:  0.10994040966033936
Epochs:  22  Loss:  0.10976787656545639
Epochs:  23  Loss:  0.10977032035589218
Epochs:  24  Loss:  0.10985516756772995
Epochs:  25  L

In [84]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0974,  0.3846],
         [-0.2103, -0.6697],
         [-0.0074,  0.6119],
         [-0.3658,  0.4935],
         [-0.5272, -0.6610],
         [-1.2038,  0.3051],
         [-0.6657, -0.4110],
         [ 0.8091,  0.2501],
         [ 0.3474,  0.3067],
         [-0.6745, -0.4262]], requires_grad=True),
 Parameter containing:
 tensor([ 1.0588,  0.2996, -0.4855,  1.0479,  0.4209, -0.0159,  0.2079,  1.0615,
          0.6247,  0.1628], requires_grad=True),
 Parameter containing:
 tensor([[ 0.4798,  0.5980, -0.0538,  0.0025, -0.1680,  0.8991, -0.3118,  0.2630,
           0.3179, -0.4398],
         [ 0.4681, -0.0047,  0.1332, -0.1919, -0.0705,  0.6795, -0.1160,  0.5245,
           0.1657, -0.0665],
         [ 0.4326, -0.5585,  0.2153,  0.5185, -0.4767,  0.5144, -0.1407, -0.0334,
           0.3198,  0.0020],
         [-0.0510, -0.0499,  0.3106,  0.0369, -0.2661, -0.2378, -0.2450, -0.0789,
          -0.1510,  0.1875],
         [ 0.1575,  0.2572, -0.3434,  0.7087,

In [102]:
data = torch.from_numpy(sc.transform([[40,20000]])).float()

In [103]:
predict = model(data)

In [104]:
predict

tensor([[-2.1857e-03, -6.1269e+00]], grad_fn=<LogSoftmaxBackward0>)

In [105]:
predicted = torch.max(predict.data, -1)
predicted

torch.return_types.max(
values=tensor([-0.0022]),
indices=tensor([0]))

In [106]:
new_predict = model(torch.from_numpy(sc.transform([[42,50000]])).float())
new_predict

tensor([[-0.7740, -0.6184]], grad_fn=<LogSoftmaxBackward0>)

In [107]:
new_predict_value = torch.max(new_predict.data, -1)

In [109]:
new_predict_value

torch.return_types.max(
values=tensor([-0.6184]),
indices=tensor([1]))

In [110]:
torch.save(model, "customer_buy.pt")

In [112]:
!ls

customer_buy.pt  sample_data


In [114]:
customer_buy_dict = model.state_dict()

In [115]:
torch.save(model.state_dict(),'customer_buy_dict')

In [116]:
!ls

customer_buy_dict  customer_buy.pt  sample_data
